In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
#from modules.models import MyRNNUAVClass
#from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv(2)
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {"model":{"vf_share_layers": True,"use_lstm": True,"max_seq_len": 100},}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {"model":{"vf_share_layers": True,"use_lstm": True,"max_seq_len": 100},}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config = ppo.DEFAULT_CONFIG.copy()
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 600*10*5,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          #"clip_actions":True,"normalize_actions":True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 300, "num_sgd_iter":20,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":600}, #PPO デフォルト "random_timesteps":0
          #"model":{"fcnet_activation": "relu","fcnet_hiddens": [256, 256, 256],"post_fcnet_activation": "linear",
          #         "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          
          #"model": {"custom_model": "my_model"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
#res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-03-07 23:25:54,330	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-03-07 23:25:54,330	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the mult

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  7 21:50:01 2022

@author: Takumi
"""
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                #for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                    #                                                   clip_actions=True,explore=False)
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j))

                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)
                
                #action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0')
                #action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1')
                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])

                # plt.clf()

                # plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
                # fig.canvas.draw()
                # plt.pause(.01)

                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue",steps,env_blue_pos_mod)
                    save_hists("red",steps,env_red_pos_mod)
                    save_hists("mrm",steps,env_mrm_pos_mod)
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            #if record_mode == 0:
               # video.release()

ray.shutdown()


----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
61 blue_0 DOWN
70 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 70 -0.1058923421189565 -2.2970674247718006
blue_1 False False 70 -1.1071256754522898 -3.057397127965445
-------------------------- Scene: 0 --------------------------
370 blue_1 DOWN
566 blue_0 Shoot at red_0 launch distance : 59737.06243646717 True True
585 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 585 -1.1109677579783446 -11.6854998054404
blue_1 False False 585 -0.10620109131167799 -9.921742280045605
-------------------------- Scene: 1 --------------------------
58 blue_1 DOWN
75 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 75 -1.1213805135002135 -2.7189839667361655
blue_1 False False 75 -0.12101384683354675 -2.784813279925883
-------------------------- Scene: 1 --------------------------
72 blue_0 DOWN
79 blue_1 DOWN
TIME LIM

76 blue_0 DOWN
449 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 449 -0.1176313943422225 -2.6520636263555364
blue_1 False False 449 -1.1125980610088892 -13.262106133317696
-------------------------- Scene: 0 --------------------------
100 blue_1 DOWN
543 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 543 -1.123942154246627 -14.759245499809909
blue_1 False False 543 -0.11810882091329356 -3.7428694957215316
-------------------------- Scene: 1 --------------------------
293 blue_1 DOWN
359 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 359 -1.120144950112127 -10.377406260949416
blue_1 False False 359 -0.11821161677879378 -14.598247085923187
-------------------------- Scene: 1 --------------------------
85 blue_0 DOWN
459 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 459 -0.11688340569263383 -2.604651895836145
blue_1 False False 459 -1.1196500723593006 -15.479678370954435
-------------------------- Scene: 0 --------------------------
422 blue_1 DOWN
428 blue_0 DOWN
TIME LIMIT LOSE
blue_0

72 blue_0 DOWN
678 blue_1 Shoot at red_1 launch distance : 59415.37393184921 True True
807 blue_1 Splash :red_1
TIME LIMIT LOSE
blue_0 False False 1200 -0.09295075550129987 -2.056816774862575
blue_1 False False 1200 0.008549244498700138 -7.867053470539672
-------------------------- Scene: 0 --------------------------
97 blue_1 DOWN
117 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 117 -1.1074149456545774 -3.4625186163801027
blue_1 False False 117 -0.10108161232124403 -4.233976864427477
-------------------------- Scene: 1 --------------------------
487 blue_0 Shoot at red_0 launch distance : 59572.237881379035 True True
546 blue_1 DOWN
612 blue_0 Splash :red_0
839 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 839 -1.1026040543630613 7.608185338422068
blue_1 False False 839 -0.09797072102972793 -13.106083182856029
-------------------------- Scene: 0 --------------------------
48 blue_1 DOWN
572 blue_0 Shoot at red_0 launch distance : 56582.51161775269 True True
574 blue_0 DOWN
TIME LIM

2022-03-08 00:12:57,618	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 61048
custom_metrics: {}
date: 2022-03-08_00-20-44
done: false
episode_len_mean: 492.3225806451613
episode_media: {}
episode_reward_max: 39.14132583838142
episode_reward_mean: -15.588834066247921
episode_reward_min: -67.90106110419809
episodes_this_iter: 62
episodes_total: 62
experiment_id: 90e68e09908c4ed184cbc04937739472
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 7.190625190734863
        entropy_coeff: 0.0
        kl: 0.041663020849227905
        model: {}
        policy_loss: 0.223805770277977
        total_loss: 5.534214973449707
        vf_explained_var: -0.09889842569828033
        vf_loss: 5.302076816558838
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 8.512490272521973
        entropy_coeff: 0.0
       

2022-03-08 00:20:45,175	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000001\checkpoint-1
2022-03-08 00:20:45,176	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': 0, '_time_total': 3280.988918066025, '_episodes_total': 62}


-------------------------- Scene: 0 --------------------------
280 blue_0 Shoot at red_0 launch distance : 59769.59853171438 True True
320 red_1 Shoot at blue_0
Same tgt shoot
349 blue_1 Shoot at red_1 launch distance : 54292.70253258211 True True
369 blue_0 Splash :red_0
436 blue_1 Splash :red_1
WIN
blue_0 False True 436 26.826922728913743 43.231500082046686
blue_1 False True 436 27.778314777843406 39.095758797643015

----------------- Training at steps:1 start! -----------------
-------------------------- Scene: 0 --------------------------
163 blue_0 DOWN
408 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 408 -0.11725218726336704 -6.13610234477753
blue_1 False False 408 -1.1147188539300337 -15.659935149757398
-------------------------- Scene: 0 --------------------------
479 blue_0 DOWN
830 blue_1 Shoot at red_0 launch distance : 59316.62484966636 True True
911 red_1 Shoot at blue_1
959 blue_1 Splash :red_0
1050blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1050 -0.10813712989

TIME LIMIT LOSE
blue_0 False False 665 -0.10884399934641649 -13.12367432473678
blue_1 False False 665 -1.0878773326797497 -4.387438070347282
-------------------------- Scene: 0 --------------------------
330 blue_1 DOWN
650 blue_0 Shoot at red_0 launch distance : 45850.889036378074 True True
676 red_0 Shoot at blue_0
694 red_1 Shoot at blue_0
769blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 769 -12.529310904253201 -27.80342074771058
blue_1 False False 769 -0.09674423758653575 -11.461928696981648
-------------------------- Scene: 1 --------------------------
91 blue_1 DOWN
140 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 140 -1.122863214669366 -4.519862366538
blue_1 False False 140 -0.11712988133603261 -4.023201191590597
-------------------------- Scene: 1 --------------------------
159 blue_1 DOWN
266 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 266 -1.1115107407193199 -8.737620584907372
blue_1 False False 266 -0.10441074071931992 -5.668107470855328
-------------------------